ttempt at double lasso regression 

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

Replace 'your_data.csv' with your actual CSV file's name and path<br>
Replace 'your_target_column' with the name of the column for the dependent variable<br>
Provide the list of column names for the independent variables

In [4]:
# Change the directory to the folder containing your file
os.chdir('/Users/anvith/Desktop/ECON 470')

# Now, you can specify your file
csv_file = 'replicationData.csv'

# The rest of your code...
target_column = 'LN_GDPPC$_2021'
independent_vars = np.array(['ln_export_area', 'longitude', 'rain_min', 'humid_max', 'low_temp', 'ln_coastline_area', 
                        'island_dum', 'islam', 'legor_fr', 'region_n', 'ln_avg_gold_pop', 'ln_avg_oil_pop', 
                        'ln_avg_all_diamonds_pop'] + [f'colony{i}' for i in range(8)])

Reading the data from CSV file into a pandas DataFrame

In [5]:
df = pd.read_csv(csv_file)

Extracting the independent variables (X) and the dependent variable (y) from the DataFrame

In [6]:
X = df[independent_vars].values
y = df[target_column].values

Splitting the dataset into training and testing sets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

First Lasso: Variable selection

In [8]:
lasso1 = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
selected_features = np.where(lasso1.coef_ != 0)[0]

Second Lasso: Estimation using selected features

In [9]:
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

In [10]:
lasso2 = LassoCV(cv=5, random_state=0).fit(X_train_selected, y_train)

Printing selected features and their coefficients

In [11]:
print("Selected Features:", independent_vars[selected_features])
print("Coefficients:", lasso2.coef_)

Selected Features: ['ln_export_area' 'longitude' 'rain_min' 'humid_max' 'low_temp'
 'ln_coastline_area' 'islam' 'ln_avg_oil_pop']
Coefficients: [-0.06795345 -0.0064646   0.0077102  -0.0069389  -0.01350543  0.08087943
 -0.00203796  0.10469955]


Optionally, using statsmodels for detailed statistics

In [12]:
X_train_sm = sm.add_constant(X_train_selected)
model_sm = sm.OLS(y_train, X_train_sm).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.668
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     6.530
Date:                Wed, 03 Apr 2024   Prob (F-statistic):           0.000112
Time:                        12:07:07   Log-Likelihood:                -28.855
No. Observations:                  35   AIC:                             75.71
Df Residuals:                      26   BIC:                             89.71
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.7166      0.922     10.542      0.0